# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Import cities.csv

csvpath = '../WeatherPy/output_data/cities.csv'
cities_df = pd.read_csv(csvpath, index_col=0)
cities_df


,City,Latitude,Longitude,Max Temperature (°F),Humidity (%),Cloudiness (%),Wind Speed (MPH),Country,Date
0,kruisfontein,-34.0033,24.7314,62.01,87,69,1.99,ZA,1612752759
1,saint george,37.1041,-113.5841,57.20,17,1,6.91,US,1612752661
2,ushuaia,-54.8000,-68.3000,50.00,81,75,4.61,AR,1612752756
3,hilo,19.7297,-155.0900,77.00,61,75,8.05,US,1612752782
4,avarua,-21.2078,-159.7750,84.20,74,20,10.36,CK,1612752783
...,...,...,...,...,...,...,...,...,...
555,port hawkesbury,45.6169,-61.3485,25.00,93,90,13.80,CA,1612753419
556,malanje,-9.5402,16.3410,63.93,97,100,7.05,AO,1612753420
557,camopi,3.1655,-52.3158,72.07,98,100,2.01,GF,1612753421
558,sunndalsora,62.6754,8.5515,19.40,68,90,4.61,NO,1612753422


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gmaps
gmaps.configure(api_key = g_key)

# Use the Lat and Lng as locations and Humidity as the weight.

locations = cities_df[["Latitude", "Longitude"]]
humidity = cities_df['Humidity (%)'].astype(float)

#add heatmap 

heat_fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating= False, max_intensity=90,point_radius=5)

heat_fig.add_layer(heat_layer)

heat_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Narrowing down the cities for the fit weather
fit_cities_df = cities_df.loc[(cities_df['Max Temperature (°F)'] > 70) &
                        (cities_df['Max Temperature (°F)'] < 80) & 
                        (cities_df['Wind Speed (MPH)'] < 10) &
                        (cities_df['Cloudiness (%)'] == 0)]

# Drop any rows will null values
hotel_df = fit_cities_df.dropna()

# Print out the number of cities and display the dataframe
print(f'The {len(hotel_df)} cities fit weather conditions')

hotel_df

The 15 cities fit weather conditions


,City,Latitude,Longitude,Max Temperature (°F),Humidity (%),Cloudiness (%),Wind Speed (MPH),Country,Date
32,moron,-34.6534,-58.6198,77.00,71,0,3.00,AR,1612752604
36,acajutla,13.5928,-89.8275,77.07,78,0,6.87,SV,1612752819
132,orodara,10.9492,-4.9342,70.74,24,0,9.71,BF,1612752927
168,tezu,27.9167,96.1667,72.10,22,0,2.80,IN,1612752847
174,kibala,9.1104,18.3463,71.60,20,0,3.53,TD,1612752976
185,san isidro,-12.1167,-77.0500,75.99,78,0,9.22,PE,1612752988
228,lazaro cardenas,17.9583,-102.2000,73.31,77,0,2.53,MX,1612753036
258,tura,25.5198,90.2201,70.75,32,0,3.42,IN,1612753070
349,santa maria,-29.6842,-53.8069,71.60,88,0,9.22,BR,1612753171
355,bonthe,7.5264,-12.5050,77.90,86,0,2.75,SL,1612753178


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
Latitude = hotel_df['Latitude'].tolist()
Longitude = hotel_df['Longitude'].tolist()

first_hotel = []
hotel_search = 'hotel'
hotel_type = "lodging"
hotel_radius = 5000
hotel_num = len(hotel_df)

# Get the first hotel in the search
for i in range(hotel_num):
    hotel_coordinates = f"{Latitude[i]}, {Longitude[i]}"


    # Set up parameters in dictionary
    par = {"location": hotel_coordinates,"types": hotel_type,
        "keyword": hotel_search,"radius": hotel_radius,"key": g_key}
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 'Try' prevents stopping the code run
    try:
        
        response = requests.get(base_url, params=par).json()
        first_hotel.append(response['results'][0]['name'])
    
    except:
        
        # Indicate no hotel was found
        first_hotel.append('No Hotel Found')
        
# After the loop, add the hotel list as a column in the dataframe
hotel_df['Hotel Name'] = first_hotel


In [6]:
# Display the dataframe
hotel_df

,City,Latitude,Longitude,Max Temperature (°F),Humidity (%),Cloudiness (%),Wind Speed (MPH),Country,Date,Hotel Name
32,moron,-34.6534,-58.6198,77.00,71,0,3.00,AR,1612752604,No Hotel Found
36,acajutla,13.5928,-89.8275,77.07,78,0,6.87,SV,1612752819,No Hotel Found
132,orodara,10.9492,-4.9342,70.74,24,0,9.71,BF,1612752927,No Hotel Found
168,tezu,27.9167,96.1667,72.10,22,0,2.80,IN,1612752847,No Hotel Found
174,kibala,9.1104,18.3463,71.60,20,0,3.53,TD,1612752976,No Hotel Found
185,san isidro,-12.1167,-77.0500,75.99,78,0,9.22,PE,1612752988,No Hotel Found
228,lazaro cardenas,17.9583,-102.2000,73.31,77,0,2.53,MX,1612753036,No Hotel Found
258,tura,25.5198,90.2201,70.75,32,0,3.42,IN,1612753070,No Hotel Found
349,santa maria,-29.6842,-53.8069,71.60,88,0,9.22,BR,1612753171,No Hotel Found
355,bonthe,7.5264,-12.5050,77.90,86,0,2.75,SL,1612753178,No Hotel Found


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [8]:
# Add marker layer ontop of heat map
heat_fig_2 = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
heat_fig_2.add_layer(markers)
heat_fig_2

Figure(layout=FigureLayout(height='420px'))